In [1]:
# colab 查看 GPU 信息
!/opt/bin/nvidia-smi

Tue May 13 12:31:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 运行后出现URL，点击获得你的认证码
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 列出你云盘里有啥
!ls "/content/drive//My Drive/"

 ch11_llm   ch12_rl   ch13_others  'Colab Notebooks'


In [4]:
# 读取云盘里的文件喽，我这里读的是 bbc-text.csv


SyntaxError: incomplete input (<ipython-input-4-ff19356f4737>, line 2)

In [ ]:
import torch

In [ ]:
tensor1 = torch.zeros(3,3)
print(tensor1)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [ ]:
tensor2 = torch.eye(5,5)
tensor2

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [ ]:
tensor3 = torch.Tensor([[1,1,4], [5,1,4]])
tensor3

tensor([[1., 1., 4.],
        [5., 1., 4.]])

In [ ]:
tensor4 = torch.randn(4,3)
tensor4

tensor([[-0.6743, -1.1520, -0.0620],
        [-0.9951,  1.0416,  0.1790],
        [-1.0329,  1.0866,  1.1500],
        [ 0.5957,  1.0303, -1.6378]])

In [ ]:
tmp = tensor1.new_ones(size = tensor4.size())
tmp

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
tensorperm = torch.randperm(114)
len(tensorperm.shape)

1

In [ ]:
tensor4 = torch.clamp(tensor4, 0, -0.5)
tensor4

tensor([[-0.5000, -0.5000, -0.5000],
        [-0.5000, -0.5000, -0.5000],
        [-0.5000, -0.5000, -0.5000],
        [-0.5000, -0.5000, -0.5000]])

In [ ]:
# 通常可以集成Module类或者torch.nn.functional编写网络层
# 前者就是完整的模型，后者更像纯粹的函数，通常会使用nn.Module
# 能够使用torch.nn.sequential来构建网络层，使用字典对添加并命名层中的神经元
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = torch.nn.Sequential(
        OrderedDict(
            [
                ("cov1", torch.nn.Conv2d(3, 32, 3, 1, 1)),
                ("relu1", torch.nn.ReLU()),
                ("pool", torch.nn.MaxPool2d(2))
            ]
        )
    )

    self.dense = torch.nn.Sequential(
        OrderedDict([
            ("dense1", torch.nn.Linear(32* 3* 3, 128)),
            ("relu2", torch.nn.ReLU()),
            ("dense2", torch.nn.Linear(128, 10))
        ])
    )


In [ ]:
import numpy as np
from torchvision.datasets import mnist
# 预处理模块
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# nn和优化器
import torch.nn.functional as F
import torch.optim as optim
from torch import nn

In [ ]:
# 设置超参数
# 参数:   模型中可以被调整和学习的参数
# 超参数: 手动设置的参数，无法自主修改
train_batch_size = 256
test_batch_size = 512
learning_rate = 0.01
lr = 0.01
num_epoches = 20
momentum = 0.5

In [ ]:
# 下载数据并且进行预处理

# 定义预处理函数，放入compose
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
# 下载数据，并对其预处理
train_dataset = mnist.MNIST('./data', train = True, transform = transform, download = True)
test_dataset = mnist.MNIST('./data', train = False, transform = transform)
# dataLoader是可迭代对象，用迭代器语法对待
train_loader = DataLoader(train_dataset, batch_size = train_batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = test_batch_size, shuffle = False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.39MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.36MB/s]


In [ ]:
# 构建网络
class WriteNumsNet(nn.Module):
  # 使用sequential将层组合到一起
  def __init__(self, in_dims, n_hidden_1, n_hidden_2, out_dim):
    super(WriteNumsNet, self).__init__()
    self.layer1 = nn.Sequential(nn.Linear(in_dims, n_hidden_1), nn.BatchNorm1d(n_hidden_1))
    self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.BatchNorm1d(n_hidden_2))
    self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))

  def forward(self, x):
    x = F.relu(self.layer1(x))
    x = F.relu(self.layer2(x))
    x =self.layer3(x)
    return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = WriteNumsNet(28*28, 300, 100, 10)
model.to(device)
print(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = lr, momentum = momentum)


cuda:0


In [ ]:
# 开始训练
losses = []
acces = []
eval_losses = []
eval_acces = []

for epoch in range(num_epoches):
  train_loss = 0
  train_acc = 0
  model.train()

  # 调参修改学习率
  # if epoch % 5 == 0:
    # optimizer.param_groups[0]['lr'] *= 0.1
  for img, label in train_loader:
    img = img.to(device)
    label = label.to(device)
    img = img.view(img.size(0), -1)
    # print("see how img look like:\n", img, "\n")

    # 前向传播
    out = model(img)
    loss = criterion(out, label)

    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 记录误差
    train_loss += loss.item()
    _, pred = out.max(1)
    num_correct = (pred == label).sum().item()
    num_correct = (pred == label).sum().item()

    # 计算准确率
    _, pred = out.max(1)
    num_correct = (pred == label).sum().item()
    acc = num_correct / img.shape[0]
    train_acc += acc

  losses.append(train_loss / len(train_loader))
  acces.append(train_acc / len(train_loader))

  # 接下来在测试集测试模型效果
  eval_loss = 0
  eval_acc = 0
  model.eval()
  for img, label in test_loader:
    img = img.to(device)
    label = label.to(device)
    img = img.view(img.size(0), -1)
    out = model(img)
    loss = criterion(out, label)
    eval_loss += loss.item()

    # 记录误差
    _, pred = out.max(1)
    num_correct = (pred == label).sum().item()
    acc = num_correct / img.shape[0]
    eval_acc += acc

  eval_losses.append(eval_loss / len(test_loader))
  eval_acces.append(eval_acc / len(test_loader))
  if epoch % 10 == 0:
    print('epoch: {}, Train Loss: {:.4f}, Train Acc: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.4f}'.format(epoch, train_loss / len(train_loader), train_acc / len(train_loader),eval_loss / len(test_loader), eval_acc / len(test_loader)))


epoch: 0, Train Loss: 0.4577, Train Acc: 0.9094, Test Loss: 0.4362, Test Acc: 0.9156
epoch: 10, Train Loss: 0.4567, Train Acc: 0.9100, Test Loss: 0.4354, Test Acc: 0.9154


In [ ]:
print(loss)

tensor(0.6510, device='cuda:0', grad_fn=<NllLossBackward0>)
